(meter-analysis)=
# Meter analysis

In [ ]:
## Importing compiam to the project
import compiam

# Import extras and supress warnings to keep the tutorial clean
import os
from pprint import pprint
import IPython.display as ipd

import warnings
warnings.filterwarnings('ignore')

(cmr-hmr)=
## Carnatic and Hindustani Music Rhyhtm datasets
That is a precise moment to introduce the (CompMusic) [Carnatic](https://zenodo.org/record/1264394) and [Hindustani](https://zenodo.org/record/1264742) Rhythm Datasets. These datasets, which include audio recordings, musically-relevant metadata, and beat and meter annotations, can be downloaded from Zenodo under request and used through the `mirdata` dataloaders available from [release 0.3.7](https://github.com/mir-dataset-loaders/mirdata/releases/tag/0.3.7).

Let's initialise an instance of these datasets and browse through the available data.

In [ ]:
cmr = compiam.load_dataset(
    "compmusic_carnatic_rhythm",
    data_home=os.path.join("../audio/mir_datasets"),
    version="full_dataset"
)
cmr

For showcasing purposes, we include a single-track version of the (CompMusic) Carnatic Rhythm Dataset within the materials of this tutorial. This dataset is private, but can be requested and downloaded for research purposes.

Reading through the dataset details we observe the available data for the tracks in the dataloader. We will load the tracks and select the specific track we have selected for tutoring purposes. Bear in mind that you can access the list of identifiers in the dataloader with the ``.track_ids`` attribute.

In [ ]:
track = cmr.load_tracks()["10001"]

Let's print out the available annotations for this example track.

In [ ]:
track.beats

``BeatData`` is an annotation type that is used in `mirata` to store information related to rhythmic beats. We will print the time-steps for the first 20 annotated beats.

In [ ]:
track.beats.times[:20]

Let's also observe the actual magnitude of these annotations.

In [ ]:
track.beats.time_unit

We have confirmed that the beats are annotated in `s`, seconds. We can also observe the positons in the cycle (if available) that each of the beats occupy.

In [ ]:
track.beats.positions[:20]

`mirdata` annotations have been created aiming at providing standardized and useful data structures for MIR-related annotations. In this example we showcase the use of ``BeatData``, but many more annotation types are included in `mirdata`. Make sure to [check them out](https://mirdata.readthedocs.io/en/latest/source/mirdata.html#annotations)!.

Let's now observe how the meter annotations looks like.

In [ ]:
track.meter

Although a list of works for metrical analysis in Indian Art Music are available {cite}`srinivasamurthy_rhythm_2014, srinivasamurthy_rhythm_2015, srinivasamurthy_rhythm_2016, srinivasamurthy_rhythm_2017a`, no tools to track the beats or the meter in Indian Art Music performances are available in `compiam` as of now. We are looking forward to covering these tasks in the near future!

Nonetheless, in this section we would like to showcase a relevant tool to track the [aksharas]() in a music recording.

(akshara-pulse-tracker)=
## Akshara pulse tracker

Let's first listen to the audio example we are going to be using to showcase this tool. We will just load the first 30 seconds (we can just slice the audio array from 0 to $f_{s} * sec$, being $sec$ the integer  number of seconds to get).

In [ ]:
# Let's listen to 30 seconds of the input audio
audio, sr = track.audio
ipd.Audio(
    data=audio[:10*sr],
    rate=sr,
)

We now import the onset detection tool from ``compiam.rhythm.meter``.

In [ ]:
# We import the tool
from compiam.rhythm.meter import AksharaPulseTracker

# Let's initialize an instance of the tool
apt = AksharaPulseTracker()

Likewise the other extractors and models in `compiam` (if not indicated otherwise), the method for inference takes an audio path as input.

In [ ]:
predicted_beats = apt.extract(track.audio_path)

Let's visualise the first 20 onset estimations.

In [ ]:
predicted_beats[:20]

We can now use the ``compiam.visualisation`` to plot the input audio with the annotations on top. 

In [ ]:
from compiam.visualisation.audio import plot_waveform
help(plot_waveform)

Let's first plot the audio waveform of the example track with the ground-truth annotations on top. We observe that the ``labels`` input variable in ``plot_waveforms`` needs to be a dict with the following format: ``{time-step: label}``, while our estimation is basically a list of pulses. Therefore, we first need to convert the prediction into a dictionary.

In [ ]:
# Converting from BeatData to {time-step: value} dict for plotting
ground_truth_beats = {
    time_step: position for time_step, position in \
        zip(track.beats.times, track.beats.positions)
}

# We then plot the first 5 seconds
plot_waveform(
    file_path=track.audio_path,
    t1=0,
    t2=5,
    labels=ground_truth_beats,
);

We can now take the predicted beats, convert these into a dictionary as well, and plot them on top of the waveform of the input signal again, in order to compare between the estimation and the ground-truth above.

In [ ]:
predicted_beats_dict = {
    time_step: idx for idx, time_step in enumerate(predicted_beats)
}

# And we plot!
plot_waveform(
    file_path=track.audio_path,
    t1=0,
    t2=4,
    labels=predicted_beats_dict,
);

In fact, `Essentia` includes a pretty cool method to create audio marks at given onsets on top of a particular input audio. Let's listen how to detected onsets sound:

In [ ]:
%pip install essentia

After installing `Essentia` in case it is not, we can define an ``AudioOnsetsMarker`` instance to mark the input audio.

In [ ]:
import essentia
import numpy as np
import essentia.standard as estd
print(np.shape(predicted_beats))
marker = estd.AudioOnsetsMarker(onsets=essentia.array(predicted_beats), type="beep")

Let's create the marked audio and listen both unmarked and marked audio signals! We only display the first 30 seconds of the entire recordings, which is notably long.

In [ ]:
ipd.Audio(
    data=audio[:10*sr],
    rate=sr,
)

In [ ]:
marked_audio = marker(audio[:10*sr])

ipd.Audio(
    data=marked_audio[:10*sr],
    rate=sr,
)